# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [6]:
## Your code here
from collections import Counter
import random
cnt = Counter(int_words)

def discard_prob(frequency, t):
    return 1-np.sqrt(t/frequency)

# The final subsampled word list
train_words = [word for word in int_words if random.random() >= discard_prob(cnt[word]/len(int_words), 1e-5)]

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [7]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    window_size = random.randint(1, window_size)
    start_index = idx-window_size if (idx-window_size) > 0 else 0
    return words[start_index:idx] + words[idx+1:idx+1+window_size]

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [8]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [9]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[None])
    labels = tf.placeholder(tf.int32, shape=[None, None])

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [10]:
n_vocab = len(int_to_vocab)
n_embedding =  200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), minval=-1, maxval=1))# create embedding weight matrix here
    embed = tf.nn.embedding_lookup(embedding, inputs)# use tf.nn.embedding_lookup to get the hidden layer output

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [11]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))# create softmax weight matrix here
    softmax_b = tf.Variable(tf.zeros(n_vocab))# create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [12]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [13]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [14]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 5.6230 0.0302 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 5.5511 0.0269 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 5.4285 0.0272 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 5.5510 0.0270 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 5.4887 0.0272 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 5.4760 0.0272 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 5.5292 0.0272 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 5.5089 0.0270 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 5.4286 0.0270 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 5.4040 0.0270 sec/batch
Nearest to four: geary, jadavpur, mexicanus, leader, lethargy, uncompleted, tumbuka, conde,
Nearest to see: tosk, stringbean, illumination, stove, stochastic, confidential, tanna, gondwana,
Nearest to all: sick, gy, triggering, strokes, shark, grumpy, lenape, downhill,
Nearest to will: cutaway, con

Epoch 1/10 Iteration: 4100 Avg. Training loss: 4.6647 0.0278 sec/batch
Epoch 1/10 Iteration: 4200 Avg. Training loss: 4.6560 0.0269 sec/batch
Epoch 1/10 Iteration: 4300 Avg. Training loss: 4.6417 0.0269 sec/batch
Epoch 1/10 Iteration: 4400 Avg. Training loss: 4.6354 0.0268 sec/batch
Epoch 1/10 Iteration: 4500 Avg. Training loss: 4.6373 0.0271 sec/batch
Epoch 1/10 Iteration: 4600 Avg. Training loss: 4.6286 0.0271 sec/batch
Epoch 2/10 Iteration: 4700 Avg. Training loss: 4.6025 0.0211 sec/batch
Epoch 2/10 Iteration: 4800 Avg. Training loss: 4.5697 0.0269 sec/batch
Epoch 2/10 Iteration: 4900 Avg. Training loss: 4.5231 0.0268 sec/batch
Epoch 2/10 Iteration: 5000 Avg. Training loss: 4.5170 0.0268 sec/batch
Nearest to four: geary, leader, gdp, laureate, erich, tumbuka, pioneer, poverty,
Nearest to see: tosk, stove, stochastic, naively, teotihuacan, tanna, esp, personal,
Nearest to all: sick, gy, ability, downhill, flowering, baltic, rite, hangings,
Nearest to will: concept, cutaway, exhorted,

Epoch 2/10 Iteration: 8100 Avg. Training loss: 4.3443 0.0278 sec/batch
Epoch 2/10 Iteration: 8200 Avg. Training loss: 4.2782 0.0269 sec/batch
Epoch 2/10 Iteration: 8300 Avg. Training loss: 4.3754 0.0269 sec/batch
Epoch 2/10 Iteration: 8400 Avg. Training loss: 4.3712 0.0267 sec/batch
Epoch 2/10 Iteration: 8500 Avg. Training loss: 4.3766 0.0268 sec/batch
Epoch 2/10 Iteration: 8600 Avg. Training loss: 4.2663 0.0267 sec/batch
Epoch 2/10 Iteration: 8700 Avg. Training loss: 4.2626 0.0267 sec/batch
Epoch 2/10 Iteration: 8800 Avg. Training loss: 4.3323 0.0268 sec/batch
Epoch 2/10 Iteration: 8900 Avg. Training loss: 4.2180 0.0269 sec/batch
Epoch 2/10 Iteration: 9000 Avg. Training loss: 4.2498 0.0269 sec/batch
Nearest to four: geary, zero, six, gdp, poverty, two, one, three,
Nearest to see: tosk, stochastic, esp, teotihuacan, pohl, visualization, ginette, depth,
Nearest to all: sick, ability, gy, hangings, prejudicial, flowering, spammers, rite,
Nearest to will: cutaway, concept, exhorted, fill,

Epoch 3/10 Iteration: 12100 Avg. Training loss: 4.1903 0.0279 sec/batch
Epoch 3/10 Iteration: 12200 Avg. Training loss: 4.1727 0.0268 sec/batch
Epoch 3/10 Iteration: 12300 Avg. Training loss: 4.1684 0.0269 sec/batch
Epoch 3/10 Iteration: 12400 Avg. Training loss: 4.1793 0.0270 sec/batch
Epoch 3/10 Iteration: 12500 Avg. Training loss: 4.1236 0.0271 sec/batch
Epoch 3/10 Iteration: 12600 Avg. Training loss: 4.0855 0.0271 sec/batch
Epoch 3/10 Iteration: 12700 Avg. Training loss: 4.1608 0.0267 sec/batch
Epoch 3/10 Iteration: 12800 Avg. Training loss: 4.0977 0.0268 sec/batch
Epoch 3/10 Iteration: 12900 Avg. Training loss: 4.1753 0.0268 sec/batch
Epoch 3/10 Iteration: 13000 Avg. Training loss: 4.2073 0.0269 sec/batch
Nearest to four: six, geary, three, two, one, zero, five, seven,
Nearest to see: tosk, stochastic, pohl, esp, teotihuacan, whetstone, italic, ginette,
Nearest to all: every, wisely, sick, hangings, prejudicial, rangle, if, points,
Nearest to will: cutaway, concept, fill, gunships

Epoch 4/10 Iteration: 16100 Avg. Training loss: 4.0120 0.0282 sec/batch
Epoch 4/10 Iteration: 16200 Avg. Training loss: 4.0370 0.0270 sec/batch
Epoch 4/10 Iteration: 16300 Avg. Training loss: 4.0696 0.0272 sec/batch
Epoch 4/10 Iteration: 16400 Avg. Training loss: 4.0022 0.0272 sec/batch
Epoch 4/10 Iteration: 16500 Avg. Training loss: 4.0320 0.0272 sec/batch
Epoch 4/10 Iteration: 16600 Avg. Training loss: 4.0327 0.0271 sec/batch
Epoch 4/10 Iteration: 16700 Avg. Training loss: 4.0079 0.0272 sec/batch
Epoch 4/10 Iteration: 16800 Avg. Training loss: 4.0375 0.0274 sec/batch
Epoch 4/10 Iteration: 16900 Avg. Training loss: 4.0538 0.0273 sec/batch
Epoch 4/10 Iteration: 17000 Avg. Training loss: 4.0408 0.0272 sec/batch
Nearest to four: six, three, one, two, five, zero, nine, seven,
Nearest to see: tosk, stochastic, whetstone, pohl, italic, esp, wipo, gj,
Nearest to all: every, wisely, points, if, sick, hangings, rangle, prejudicial,
Nearest to will: cutaway, fill, either, concept, grisman, ciph

...

Epoch 7/10 Iteration: 28100 Avg. Training loss: 3.8674 0.0284 sec/batch
Epoch 7/10 Iteration: 28200 Avg. Training loss: 3.8389 0.0275 sec/batch
Epoch 7/10 Iteration: 28300 Avg. Training loss: 3.8323 0.0270 sec/batch
Epoch 7/10 Iteration: 28400 Avg. Training loss: 3.8544 0.0273 sec/batch
Epoch 7/10 Iteration: 28500 Avg. Training loss: 3.7953 0.0273 sec/batch
Epoch 7/10 Iteration: 28600 Avg. Training loss: 3.9047 0.0272 sec/batch
Epoch 7/10 Iteration: 28700 Avg. Training loss: 3.8375 0.0273 sec/batch
Epoch 7/10 Iteration: 28800 Avg. Training loss: 3.9059 0.0274 sec/batch
Epoch 7/10 Iteration: 28900 Avg. Training loss: 3.8114 0.0273 sec/batch
Epoch 7/10 Iteration: 29000 Avg. Training loss: 3.8347 0.0274 sec/batch
Nearest to four: six, one, two, five, three, zero, nine, seven,
Nearest to see: of, external, whetstone, article, links, wipo, stochastic, pohl,
Nearest to all: the, only, every, to, if, their, of, wisely,
Nearest to will: cutaway, switching, dischord, grisman, concept, rushed, d

Epoch 7/10 Iteration: 32100 Avg. Training loss: 3.8546 0.0283 sec/batch
Epoch 7/10 Iteration: 32200 Avg. Training loss: 3.8592 0.0274 sec/batch
Epoch 7/10 Iteration: 32300 Avg. Training loss: 3.8307 0.0273 sec/batch
Epoch 8/10 Iteration: 32400 Avg. Training loss: 3.9184 0.0077 sec/batch
Epoch 8/10 Iteration: 32500 Avg. Training loss: 3.8392 0.0272 sec/batch
Epoch 8/10 Iteration: 32600 Avg. Training loss: 3.8210 0.0273 sec/batch
Epoch 8/10 Iteration: 32700 Avg. Training loss: 3.8464 0.0273 sec/batch
Epoch 8/10 Iteration: 32800 Avg. Training loss: 3.8490 0.0272 sec/batch
Epoch 8/10 Iteration: 32900 Avg. Training loss: 3.7825 0.0273 sec/batch
Epoch 8/10 Iteration: 33000 Avg. Training loss: 3.8309 0.0272 sec/batch
Nearest to four: six, one, two, five, three, zero, seven, nine,
Nearest to see: external, of, list, article, links, whetstone, disambiguation, wipo,
Nearest to all: the, only, of, to, their, every, other, is,
Nearest to will: cutaway, switching, concept, rushed, grassmann, unclea

Epoch 8/10 Iteration: 36100 Avg. Training loss: 3.9335 0.0284 sec/batch
Epoch 8/10 Iteration: 36200 Avg. Training loss: 3.8718 0.0272 sec/batch
Epoch 8/10 Iteration: 36300 Avg. Training loss: 3.8364 0.0273 sec/batch
Epoch 8/10 Iteration: 36400 Avg. Training loss: 3.7677 0.0273 sec/batch
Epoch 8/10 Iteration: 36500 Avg. Training loss: 3.8085 0.0272 sec/batch
Epoch 8/10 Iteration: 36600 Avg. Training loss: 3.7545 0.0273 sec/batch
Epoch 8/10 Iteration: 36700 Avg. Training loss: 3.8089 0.0272 sec/batch
Epoch 8/10 Iteration: 36800 Avg. Training loss: 3.8243 0.0273 sec/batch
Epoch 8/10 Iteration: 36900 Avg. Training loss: 3.7978 0.0271 sec/batch
Epoch 8/10 Iteration: 37000 Avg. Training loss: 3.8334 0.0275 sec/batch
Nearest to four: six, one, two, three, five, zero, seven, nine,
Nearest to see: external, of, list, links, article, gj, disambiguation, whetstone,
Nearest to all: the, to, of, only, a, their, other, every,
Nearest to will: switching, cutaway, grassmann, concept, fetching, paging,

Epoch 9/10 Iteration: 40100 Avg. Training loss: 3.8000 0.0284 sec/batch
Epoch 9/10 Iteration: 40200 Avg. Training loss: 3.8046 0.0273 sec/batch
Epoch 9/10 Iteration: 40300 Avg. Training loss: 3.7666 0.0273 sec/batch
Epoch 9/10 Iteration: 40400 Avg. Training loss: 3.7772 0.0273 sec/batch
Epoch 9/10 Iteration: 40500 Avg. Training loss: 3.8910 0.0272 sec/batch
Epoch 9/10 Iteration: 40600 Avg. Training loss: 3.7904 0.0272 sec/batch
Epoch 9/10 Iteration: 40700 Avg. Training loss: 3.9237 0.0274 sec/batch
Epoch 9/10 Iteration: 40800 Avg. Training loss: 3.8614 0.0272 sec/batch
Epoch 9/10 Iteration: 40900 Avg. Training loss: 3.8436 0.0271 sec/batch
Epoch 9/10 Iteration: 41000 Avg. Training loss: 3.7632 0.0275 sec/batch
Nearest to four: one, six, two, five, three, zero, seven, nine,
Nearest to see: external, list, links, of, article, disambiguation, by, from,
Nearest to all: the, of, to, only, their, other, for, is,
Nearest to will: switching, helpers, cutaway, grassmann, fetching, grisman, asym

Epoch 10/10 Iteration: 44100 Avg. Training loss: 3.7775 0.0282 sec/batch
Epoch 10/10 Iteration: 44200 Avg. Training loss: 3.8061 0.0274 sec/batch
Epoch 10/10 Iteration: 44300 Avg. Training loss: 3.7913 0.0272 sec/batch
Epoch 10/10 Iteration: 44400 Avg. Training loss: 3.7562 0.0271 sec/batch
Epoch 10/10 Iteration: 44500 Avg. Training loss: 3.7949 0.0272 sec/batch
Epoch 10/10 Iteration: 44600 Avg. Training loss: 3.7941 0.0273 sec/batch
Epoch 10/10 Iteration: 44700 Avg. Training loss: 3.8040 0.0271 sec/batch
Epoch 10/10 Iteration: 44800 Avg. Training loss: 3.7808 0.0271 sec/batch
Epoch 10/10 Iteration: 44900 Avg. Training loss: 3.7645 0.0271 sec/batch
Epoch 10/10 Iteration: 45000 Avg. Training loss: 3.7750 0.0271 sec/batch
Nearest to four: six, one, two, three, five, zero, seven, nine,
Nearest to see: of, list, external, links, article, the, also, by,
Nearest to all: the, to, other, of, and, only, their, a,
Nearest to will: switching, helpers, cutaway, fetching, whirl, protected, grassman

Restore the trained network if you need to:

In [15]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

INFO:tensorflow:Restoring parameters from checkpoints/text8.ckpt


## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)